In [15]:
import pandas as pd
import numpy as np 
import glob
from Bio.PDB import *
from biopandas.pdb import PandasPdb

In [2]:
uniprot = "P78344"

In [5]:
parser = PDBParser()
pdb_path = "/rcfs/projects/proteometer/alphafold_swissprot_pdb"
pdb_file = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "*" ".pdb")
ppdb = PandasPdb()
ppdb.read_pdb(pdb_file[0])

print(pdb_file)


['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-P78344-F1-model_v4.pdb']


In [8]:
ppdb.df['ATOM'].head()

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,MET,,A,1,,...,-4.046,8.284,-2.813,1.0,31.08,,,N,NaN,116
1,ATOM,2,,CA,,MET,,A,1,,...,-4.995,7.749,-1.812,1.0,31.08,,,C,NaN,117
2,ATOM,3,,C,,MET,,A,1,,...,-6.000,6.928,-2.590,1.0,31.08,,,C,NaN,118
3,ATOM,4,,CB,,MET,,A,1,,...,-5.736,8.872,-1.057,1.0,31.08,,,C,NaN,119
4,ATOM,5,,O,,MET,,A,1,,...,-6.832,7.518,-3.262,1.0,31.08,,,O,NaN,120


In [7]:
atom_1 = ppdb.df['ATOM'].query('residue_number == 2 and atom_name == "CA"')
atom_2 = ppdb.df['ATOM'].query('residue_number == 100 and atom_name == "CA"')
x_p, y_p, z_p = atom_1['x_coord'].values[0], atom_1['y_coord'].values[0], atom_1['z_coord'].values[0]
x_q, y_q, z_q  = atom_2['x_coord'].values[0], atom_2['y_coord'].values[0], atom_2['z_coord'].values[0]
distance = np.sqrt((x_p - x_q)**2 + (y_p - y_q)**2 + (z_p - z_q)**2)

distance

np.float64(36.35657313334137)

In [6]:
def get_pairwise_distance(structure, residue_num_1, residue_num_2):
    atom_1 = structure.query('residue_number == @residue_num_1 and atom_name == "CA"') # restrict to only res #1 and alpha carbon 
    atom_2 = structure.query('residue_number == @residue_num_2 and atom_name == "CA"') # restrict to only res #2 and alpha carbon 
    x_p, y_p, z_p = atom_1['x_coord'].values[0], atom_1['y_coord'].values[0], atom_1['z_coord'].values[0]
    x_q, y_q, z_q  = atom_2['x_coord'].values[0], atom_2['y_coord'].values[0], atom_2['z_coord'].values[0]
    distance = np.sqrt((x_p - x_q)**2 + (y_p - y_q)**2 + (z_p - z_q)**2)
    return distance


In [7]:
input_struct = ppdb.df['ATOM']

output = get_pairwise_distance(input_struct,1,10)
print(output)
output * 2

14.348114196646192


np.float64(28.696228393292383)

In [8]:
def find_mean_distances(structure, target_residue, residue_list):
    residue1 = target_residue
    distances = []
    for residue2 in residue_list:
        residue2 = float(residue2)
        distance_to_add = get_pairwise_distance(structure, residue1, residue2)
        # print(distance_to_add)
        distances.append(distance_to_add)
        distances = [x for x in distances if x != 'NaN']
    return np.mean(distances)

In [11]:
pocket_residues = ['97', '99', '100', '101', '102', '159', '163', '189', '190']
output = find_mean_distances(input_struct,2,pocket_residues)

NameError: name 'input_struct' is not defined

In [10]:

def find_mean_distances(structure, target_residue, residue_list):
    residue1 = target_residue
    distances = []
    for residue2 in residue_list:
        residue2 = float(residue2)
        distance_to_add = get_pairwise_distance(structure, residue1, residue2)
        # print(distance_to_add)
        distances.append(distance_to_add)
        distances = [x for x in distances if x != 'NaN']
    return np.mean(distances)

def get_pairwise_distance(structure, residue_num_1, residue_num_2):
    atom_1 = structure.query('residue_number == @residue_num_1 and atom_name == "CA"') # restrict to only res #1 and alpha carbon 
    atom_2 = structure.query('residue_number == @residue_num_2 and atom_name == "CA"') # restrict to only res #2 and alpha carbon 
    if (atom_1['x_coord'].values[0], atom_1['y_coord'].values[0], atom_1['z_coord'].values[0],atom_2['x_coord'].values[0], atom_2['y_coord'].values[0], atom_2['z_coord'].values[0]):
        print(atom_1['x_coord'].values[0], atom_1['y_coord'].values[0], atom_1['z_coord'].values[0])
        print(atom_2['x_coord'].values[0], atom_2['y_coord'].values[0], atom_2['z_coord'].values[0])
        x_p, y_p, z_p = atom_1['x_coord'].values[0], atom_1['y_coord'].values[0], atom_1['z_coord'].values[0]
        x_q, y_q, z_q  = atom_2['x_coord'].values[0], atom_2['y_coord'].values[0], atom_2['z_coord'].values[0]
        distance = np.sqrt((x_p - x_q)**2 + (y_p - y_q)**2 + (z_p - z_q)**2)
    return distance

In [14]:
input_struct = ppdb.df['ATOM']
residue_num = 95.0
new_dist = find_mean_distances(input_struct, residue_num, pocket_residues)


17.355 -29.54 -10.985
17.512 -24.141 -12.244
17.355 -29.54 -10.985
13.793 -26.878 -15.078
17.355 -29.54 -10.985
13.574 -23.005 -15.129
17.355 -29.54 -10.985
15.92 -22.641 -18.172
17.355 -29.54 -10.985
13.83 -25.354 -19.951
17.355 -29.54 -10.985
30.29 -8.651 -21.745
17.355 -29.54 -10.985
19.759 -10.89 -18.101
17.355 -29.54 -10.985
6.268 16.794 10.653
17.355 -29.54 -10.985
3.603 15.267 12.959
